Clandescriptorius
=================

## Challenge
Secret writing, secret writing, secret secret secret secret writing

https://clandescriptorius.challs.pwnoh.io

## Included files
* clandescriptorius.zip

In [1]:
%%bash
unzip clandescriptorius.zip -d extracted

Archive:  clandescriptorius.zip


replace extracted/clandescriptorius/main.py? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


CalledProcessError: Command 'b'unzip clandescriptorius.zip -d extracted\n'' returned non-zero exit status 1.

## Analysis
A wen service is presented from the application.
When one posts to /startsession, a json object with a session id and the encrypted flag are returned. It seems that the timestamp here is in the attacker's control.

When one posts to /encrypt, they provide the session id, a new timestamp and data to encrypt. The timestamp must be ge the previous.

The encryption is performed by breaking up the input data into 16 byte blocks, with a key generated by concatenating the key, timestamp and block index.
Because there are no separators between these, the keystream for timestamp 1 and block 11 is equivalent to timestamp 11 and block 1.

In [ ]:
from http.client import *
import json

conn = HTTPSConnection("clandescriptorius.challs.pwnoh.io")
conn.request("POST", "/startsession", json.dumps({"timestamp": 1}))
with conn.getresponse() as response:
    session = json.loads(response.read())
print(session)

for i in range(2, 5):
    conn.request("POST", "/encrypt", json.dumps({"session_id": session["session_id"], "timestamp": i, "data": "deadbeef"}))
    with conn.getresponse() as response:
        print(json.loads(response.read()))

{'session_id': '7c35e4ff94e541ed16afe46027e97ff2', 'encrypted_flag': '8aa4e1ec2eaf102b0301c78b40d3621bda32a43354b05d99dc4f3b43648e70f1614ec4e410a45ad498c2854dd00e15d6'}
{'encrypted': '15291a54cc52805f204e1b71e288632f'}
{'encrypted': '7949d52e9362a53a13c258857eda3fe8'}
{'encrypted': '99fc7c8c0696753077156cb85da87941'}


As the encrypted flag is 48 bytes, we know that it must be split into 3 blocks i.e. 0, 1, 2.
After some brain thinking, trying to exploit the contatentation of the keys, I realised that there is no way to rebracket the timestamp/block combination where the new timestamp is less than the old, while having the same block, until I realised that there's no constraint that the timestamp be positive.

Hence a timestamp -111 and block 0 is equivalent to timestamp -11 and block 10 and -11 > -111

In [4]:
from http.client import *
from pwn import *
import json

conn = HTTPSConnection("clandescriptorius.challs.pwnoh.io")
conn.request("POST", "/startsession", json.dumps({"timestamp": -111}))
with conn.getresponse() as response:
    session = json.loads(response.read())

padding = "f0" * 16 * 10
conn.request("POST", "/encrypt", json.dumps({"session_id": session["session_id"], "timestamp": -11, "data": padding + session["encrypted_flag"]}))
with conn.getresponse() as response:
    decrypted = json.loads(response.read())

flag = unhex(decrypted["encrypted"][(16*10*2):])

print(flag[:flag.index(b"}")+1].decode())

bctf{the_future_is_now_e3faa77c672e6d62}
